# COPT and LOLP Calculator

## import and settting printer

In [1]:
import itertools
import numpy as np
import pandas as pd

In [2]:
# setting numpy print option decimal places
decimal_places = 3
np.set_printoptions(precision=decimal_places, suppress=True)
pd.set_option('display.float_format', '{:.3f}'.format)

## inputs

### choose between using csv file or manually edit the notebook

In [3]:
# # NOTE: UNCOMMENT AND USE THIS CELL IF USING CSV INPUT
# # csv input path
# df = pd.read_csv('../data/COPT Case.csv')

# # extract csv
# capacities = df['Capacity (MW)'].tolist()
# outage_rates = df['FOR (Outage)'].tolist()
# status = [True] * len(df.index)

In [4]:
# NOTE: UNCOMMENT AND USE THIS CELL IF USING MANUAL INPUT
# max capacity
capacities = [
    80,
    70,
    40,
    50,
    40,
    50
]

# forced outage rate
outage_rates = [
    0.1, 
    0.2,
    0.1,
    0.2,
    0.1,
    0.1
]


# status of generator,
#     True: exist
#     False: not exist
status = [
    True,
    False,
    True,
    True,
    True,
    True,
]

In [5]:
pd.DataFrame(data={'capacities': capacities,
                   'outage_rates': outage_rates,
                   'status': status},
             index=pd.RangeIndex(1, len(capacities) + 1, 1))

,capacities,outage_rates,status
1,80,0.100,True
2,70,0.200,False
3,40,0.100,True
4,50,0.200,True
5,40,0.100,True
6,50,0.100,True


In [6]:
# peak load or load at time considering
# used to calculate LOLP
demands = [90, 100, 125, 50]

## creating data

In [7]:
ls = [[cap, out] for cap, out, stat in sorted(zip(capacities, outage_rates, status), reverse=True) if stat]
capacities_fix = [[cap, 0] for cap, _ in ls]
probabilities_fix = [[1-out, out] for _, out in ls]

# TODO: Avoid using list here as the generator can produces very high number of combinations
capacities_combination = list(itertools.product(*capacities_fix))
probabilities_combinations = list(itertools.product(*probabilities_fix))

[[80, 0], [50, 0], [50, 0], [40, 0], [40, 0]]
[[0.9, 0.1], [0.8, 0.2], [0.9, 0.1], [0.9, 0.1], [0.9, 0.1]]


## COPT table

In [9]:
capacity_merged = np.array([sum(i) for i in capacities_combination])
probability_merged = np.array([np.prod(i) for i in probabilities_combinations])

# combine capacity and individual probability
table = np.vstack([capacity_merged, probability_merged]).T

# sort the combined table
table = table[(-table[:, 0]).argsort(),:]

# combine duplicate
table = np.array([[k,sum([x[1] for x in list(g)])] 
                 for k,g in itertools.groupby(table, lambda x:x[0])])

# add cumulative probability
table = np.hstack((table, np.atleast_2d(np.cumsum(table[::-1,1])[::-1]).T))

## display COPT table

In [10]:
columns_name = ['Combined Capacity', 'Individual Probability', 'Cumulative Probability']

In [11]:
pd.DataFrame(data=table, columns=columns_name, index=pd.RangeIndex(1, len(table) + 1, 1))

,Combined Capacity,Individual Probability,Cumulative Probability
1,260.000,0.525,1.000
2,220.000,0.117,0.475
3,210.000,0.190,0.358
4,180.000,0.065,0.169
5,170.000,0.042,0.104
6,160.000,0.015,0.062
7,140.000,0.013,0.047
8,130.000,0.023,0.034
9,120.000,0.003,0.011
10,100.000,0.001,0.008


## LOLP

In [12]:
def get_lolp(capacity, cumulative_probability, demand):
    """
    format:
        capacity (descend)
        cumulative_probability(descend)
    """
    idx = np.argmax(capacity < demand)
    return cumulative_probability[idx]

In [13]:
lolp = [get_lolp(table[:,0], table[:,2], demand) for demand in demands]
print(lolp)

[0.0024400000000000003, 0.007120000000000002, 0.011080000000000003, 0.0003800000000000001]


In [14]:
print(f'Total LOLP: {sum(lolp):.4f}')

Total LOLP: 0.0210
